# BioExplorer - Growing neurons
![](../../bioexplorer_neurons_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, NeuronDisplacementParams, Vector2, MovieMaker

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
mm = MovieMaker(be)
status = be.reset_scene()

In [ ]:
status = core.set_camera(
    orientation=[-0.12981074809058107, 0.3782067201813325, 0.0036557851586879464, 0.9165672270623232],
    position=[679.8890979521727, 1936.9563865502405, 444.8515526686998],
    target=[384.5328257698453, 1834.2635155326673, 154.67286704500805],
)
status = core.set_renderer()

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name, palette_size):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, palette_size)

    for material_id in material_ids:
        mid = material_id % palette_size
        colors.append(palette[mid])
        opacities.append(1.0)
        shading_modes.append(be.SHADING_MODE_PERLIN)
        glossinesses.append(0.1)
        user_params.append(0.05)
        specular_exponents.append(5.0)
        emissions.append(0.0)
        refraction_indices.append(1.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [ ]:
status = core.set_renderer(
    background_color=[1,1,1],
    current='advanced',subsampling=4, max_accum_frames=64)
params = core.AdvancedRendererParams()
params.gi_distance = 100.0
params.shadows = 0.0
params.soft_shadows = 1.0
params.epsilon_factor = 100.0
params.max_bounces = 1
params.show_background = False
params.exposure = 1.1
status = core.set_renderer_params(params)
status = core.set_renderer()

### Load neurons

In [ ]:
from tqdm import tqdm
for i in tqdm(range(325)):
    assembly_name = 'Neurons'
    be.remove_assembly(assembly_name)
    neurons_assembly = be.add_assembly(assembly_name)

    neurons_model = be.add_neurons(
        assembly_name=assembly_name,
        population_name='o1',
        morphology_representation=be.MORPHOLOGY_REPRESENTATION_SEGMENT,
        morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
        realism_level=be.MORPHOLOGY_REALISM_LEVEL_ALL,
        synapses_type=be.NEURON_SYNAPSES_AFFERENT, generate_varicosities=True,
        load_somas=True, load_axon=True, show_membrane=True,
        load_basal_dendrites=True, load_apical_dendrites=True,
        sql_node_filter='guid=49',
        displacement_params=NeuronDisplacementParams(
            soma=Vector2(0.015, 2.0), section=Vector2(0.1, 3.0),
            nucleus=Vector2(0.02, 1.0), mitochondrion=Vector2(0.4, 1.0),
            myelin_steath=Vector2(0.1, 4.0), spine=Vector2(0.01, 5.0)),
        max_distance_to_soma=i + 1.0
    )
    model_ids = be.get_model_ids()['ids']
    set_morphology_materials(model_ids[0], 'Set3', be.NB_MATERIALS_PER_MORPHOLOGY)
    mm.create_snapshot(
        renderer='advanced',
        path='/tmp',
        base_name='%05d' % i,
        size=[1024, 1024], samples_per_pixel=16)
    core.set_renderer()